In [5]:
from sqlalchemy import create_engine
from sqlalchemy.sql import text

# Replace these with your installation details
MARIADB_USER = 'oltp_user'
MARIADB_HOST = '127.0.0.1' # localhost
MARIADB_PORT = '3306'
MARIADB_PASS = 'oltp_pass'
MARIADB_DB = 'oltp'

CONNECTION_URI = f'mysql+pymysql://{MARIADB_USER}:{MARIADB_PASS}@{MARIADB_HOST}:{MARIADB_PORT}/{MARIADB_DB}'

# Create the SQLAlchemy engine object. This will provide connections to MariaDB.
# The echo=True keyword argument will make SQLAlchemy print its activity. Use echo=False to stop it.
engine = create_engine(CONNECTION_URI, echo=True, future=True)

In [9]:
# Test the connection by just executing the SQL statement "SELECT 1;"
# This is the "hello world" equivalent for databases

# This is a context manager. It will automatically close the connection once you leave its scope.
with engine.connect() as conn:
    query = '''SELECT 1;''' # This is just a SQL query expressed in a Python string
    stmt = text(query) # This is wrapping a string query in a special "text" object from SQLAlchemy. This will be important later.
    response = conn.execute(stmt) # Send the query to the connection and make MariaDB execute your query. Save the result cursor in a `response` variable
    # conn.commit() # Use this line to persist the changes you made to the database. No need to do it here, but this will be useful later.
    
print(response.all()) # Use response.all() to fetch all the objects from the response.

2022-06-20 10:52:03,123 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-06-20 10:52:03,125 INFO sqlalchemy.engine.Engine SELECT 1;
2022-06-20 10:52:03,128 INFO sqlalchemy.engine.Engine [cached since 400.1s ago] {}
2022-06-20 10:52:03,129 INFO sqlalchemy.engine.Engine ROLLBACK
[(1,)]


In [11]:
# Create a table for users called `user` (because every row is about a user)

with engine.connect() as conn:
    query = '''
    CREATE TABLE IF NOT EXISTS user (
        id INTEGER AUTO_INCREMENT PRIMARY KEY,
        first_name TEXT,
        last_name TEXT,
        email TEXT
    );
    '''
    stmt = text(query)
    conn.execute(stmt) # This returns nothing, so don't capture a response
    conn.commit()

2022-06-20 10:56:55,840 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-06-20 10:56:55,841 INFO sqlalchemy.engine.Engine 
    CREATE TABLE IF NOT EXISTS user (
        id INTEGER AUTO_INCREMENT PRIMARY KEY,
        first_name TEXT,
        last_name TEXT,
        email TEXT
    );
    
2022-06-20 10:56:55,842 INFO sqlalchemy.engine.Engine [generated in 0.00192s] {}
2022-06-20 10:56:55,844 INFO sqlalchemy.engine.Engine COMMIT


In [16]:
# Add some values to the `user` table dynamically

users = [
    {'first_name': 'Matthew', 'last_name': 'Uy', 'email': 'matthew.uy@domain.com'},
    {'first_name': 'Aly', 'last_name': 'Yap', 'email': 'aly.yap@domain.com'},
    {'first_name': 'Bong', 'last_name': 'Olpoc', 'email': 'bong.olpoc@domain.com'},
]

with engine.connect() as conn:
    for user in users:
        query = '''
        INSERT INTO user (first_name, last_name, email)
        VALUES
        (:first_name, :last_name, :email);
        ''' # These tokens with prefixed colons are stand-ins for values that you will pass later.
        stmt = text(query)
        conn.execute(stmt, user) # Pass a dictionary whose keys are the parameters from the query and whose values are the actual values you want to insert.
    conn.commit()

2022-06-20 11:05:00,083 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-06-20 11:05:00,085 INFO sqlalchemy.engine.Engine 
        INSERT INTO user (first_name, last_name, email)
        VALUES
        (%(first_name)s, %(last_name)s, %(email)s);
        
2022-06-20 11:05:00,086 INFO sqlalchemy.engine.Engine [generated in 0.00383s] {'first_name': 'Matthew', 'last_name': 'Uy', 'email': 'matthew.uy@domain.com'}
2022-06-20 11:05:00,089 INFO sqlalchemy.engine.Engine 
        INSERT INTO user (first_name, last_name, email)
        VALUES
        (%(first_name)s, %(last_name)s, %(email)s);
        
2022-06-20 11:05:00,090 INFO sqlalchemy.engine.Engine [cached since 0.007468s ago] {'first_name': 'Aly', 'last_name': 'Yap', 'email': 'aly.yap@domain.com'}
2022-06-20 11:05:00,092 INFO sqlalchemy.engine.Engine 
        INSERT INTO user (first_name, last_name, email)
        VALUES
        (%(first_name)s, %(last_name)s, %(email)s);
        
2022-06-20 11:05:00,094 INFO sqlalchemy.engine.Engine [

In [20]:
# Query the database

with engine.connect() as conn:
    query = '''SELECT * FROM user WHERE email = 'bong.olpoc@domain.com';'''
    stmt = text(query)
    res = conn.execute(stmt)

results = res.first()
print()
# Print the whole row
print(results)
# Print only one attribute. The row is a named tuple so you can access attributes by name.
print(results.first_name)

2022-06-20 11:08:23,048 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-06-20 11:08:23,051 INFO sqlalchemy.engine.Engine SELECT * FROM user WHERE email = 'bong.olpoc@domain.com';
2022-06-20 11:08:23,052 INFO sqlalchemy.engine.Engine [cached since 70.25s ago] {}
2022-06-20 11:08:23,055 INFO sqlalchemy.engine.Engine ROLLBACK

(3, 'Bong', 'Olpoc', 'bong.olpoc@domain.com')
Bong


In [22]:
# A sample interactive cell for people to create their own users
# This assumes that your objects (e.g., engine) already exist

print('Welcome to the system!')

while True:
    res = input("Would you like to create a user? (y/n)")
    if res == 'n':
        print('Exiting system.')
        break
    # Gather information
    first_name = input('What is your first name?')
    last_name = input('What is your last name?')
    email = input('What is your email address?')
    # Store information in a dictionary
    user = {'first_name': first_name, 'last_name': last_name, 'email': email}
    # Generate query
    query = '''
    INSERT INTO user (first_name, last_name, email)
    VALUES
    (:first_name, :last_name, :email);
    '''
    stmt = text(query)
    # Execute query
    with engine.connect() as conn:
        conn.execute(stmt, user)
        conn.commit()


Welcome to the system!
2022-06-20 11:18:29,417 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-06-20 11:18:29,420 INFO sqlalchemy.engine.Engine 
    INSERT INTO user (first_name, last_name, email)
    VALUES
    (%(first_name)s, %(last_name)s, %(email)s);
    
2022-06-20 11:18:29,422 INFO sqlalchemy.engine.Engine [generated in 0.00465s] {'first_name': 'Joe', 'last_name': 'Ilagan', 'email': 'joe.ilagan@domain.com'}
2022-06-20 11:18:29,425 INFO sqlalchemy.engine.Engine COMMIT
Exiting system.
